<a href="https://colab.research.google.com/github/ludawg44/jigsawlabs/blob/master/16Apr20_2_a_data_paradise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A perfect world

### Introduction

In the last lesson, we discussed the different types of error that we see when we train our machine learning models: irreducible error, variance and bias.  In the lessons that follow, we'll explore these errors in turn.  

Our technique for exploring errors is to first construct our data in such a way that these errors do not exist.  We'll do so for two reasons:

1. The easiest way to appreciate and visualize our different errors is to see what a machine learning model looks like when they do not exist.
2. The ability to create artificial datasets is crucial to our ability to explore new techniques and algorithms


So in this lesson, we'll create a dataset that our machine learning model can perfectly predict.  Then we'll train our model on this dataset.

Let's get going.

### Imagining a world without error

Here's our scenario.

Imagine that we are hired by a restaurant to predict the number of customers who enter our restaurant each day.  The restaurant wants to reduce the amount it wastes on purchasing extra food supplies.  It knows that temperature affects the number of customers it sees each day, but it doesn't know by how much.

In other words, our model looks like the following:

$$ customers = \theta_1*temperature + \theta_0 $$

> Here we use the variable theta, $\theta$ to represent the parameter for temperature that we are trying top discover.  Previously, we have used $x_1$ or $m$ for this.  $\theta_1$ is equivalent to our coefficient parameter and $\theta_0$ is equivalent to our intercept parameter.

Now we don't have any data yet from the restaurant -- they haven't yet hired us -- so we need to create a mock dataset ourselves.  Let's first explore how to create a synthetic dataset, and then we'll see what our model from this data looks like.

### 1. Creating random numbers

We know that our only feature is temperature and our target is the number of customers.  So we can start off by feature variable data -- a list of random temperatures between 30 degrees and 101 degrees.

In [0]:
import numpy as np
import numpy.random as random

In [0]:

random_temperatures = random.randint(30, 101, 50)
random_temperatures[:3]

array([50, 69, 78])

> Press shift + enter on the cell above multiple times.  

Notice that every time the cell is executed, a different set of numbers is generated.  This makes sense, but it can also be problematic.  We don't want this process to generate a list of random numbers that we cannot reproduce.  What if something goes wrong?  We'll need the ability to recreate our data so that we can then carefully explore everything that went wrong.  

So we want the ability to generate the same list of random, really pseudo-random numbers, each time.  We do this by *seeding* our random number generator. 

Let's see how seeding works.

In [0]:
random.seed(2)
random_temperatures = random.randint(30, 101, 50)
random_temperatures[:3]

array([70, 45, 52])

Seeding configures our random number generator to generate the same sequence of numbers.  Try re-executing the cell above, notice that the beginning numbers above are now always 70, 45, and 52.

### 2. Generate perfect outcomes

Ok, so now that we have generated a list of random temperatures, our feature data, it's time to generate our target data or labels.  

Remember our goal is to construct a perfect dataset.  Also remember that our the restaurant believes that the true model follows the following form: 

$$ customers = \theta_1*temperature + \theta_0 $$

Let's assume that $\theta_1 = 3$ and $\theta_0 = 10$.  We then have the following model:

$$ customers = 3*temperature + 10 $$

And we should be able to construct a dataset that perfectly matches this model.  Here it is.

> In Python

In [0]:
perfect_customers = [3*temp + 10 for temp in random_temperatures]
perfect_customers[0:3]

[220, 145, 166]

> Or in Numpy

In [0]:
perfect_customers = 3*random_temperatures + 10
perfect_customers[:3]

array([220, 145, 166])

Ok, so let's plot our data.

In [0]:
import plotly.graph_objects as go
# from graph import trace_values, plot
data_trace = go.Scatter(x = random_temperatures, y = perfect_customers, 
                        name = 'initial data', mode = 'markers')
layout = {'yaxis': {'title': 'customers'}, 'xaxis': {'title': 'temperature'}}
go.Figure([data_trace], layout = layout)

### 3. Training our linear model

Now so far we created a synthetic dataset.  And to do so, we started off with our true underlying model, and then created our dataset from there.

But normally, we do not know the true underlying model.  Instead we use a machine learning algorithm to approximate it.  Let's imagine we did not know that our underlying model was $ customers = 3x + 10$ and see how good our regression model is approximating this underlying model.

1. Transform dataset into rows

SciKit needs our feature variables to have a separate array for each row of data.  So we reshape our data accordingly.

In [0]:
random_temperatures[:3]

array([70, 45, 52])

In [0]:
reshaped_temps = random_temperatures.reshape(-1, 1)
reshaped_temps[:3]

array([[70],
       [45],
       [52]])

2. Fit the model and view the estimated parameters

Next we go through our standard steps of training our linear regression model, to see if the model can detect the underlying pattern in the data.

In [0]:
from sklearn.linear_model import LinearRegression
perfect_model = LinearRegression()
perfect_model.fit(reshaped_temps, perfect_customers)

perfect_model.coef_

array([3.])

In [0]:
perfect_model.intercept_

9.999999999999886

Hot damn! Our regression model started off not knowing the underlying true model, but by feeding it data, it was able to discover this model.  

We can confirm that this model matches the data well, by scoring the model against this data with the root mean squared error.

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [0]:
sqrt(mean_squared_error(perfect_customers, perfect_model.predict(reshaped_temps)))

3.475127886257918e-14

That number is a very small number, essentially 0.

### 4. Predicting with our model

Now that this model has been trained on our data, we can try the model on data it has not yet seen.  Because we have a world where the underlying model has $customers =  3*temp + 10 $, and this what our model predicts, our model will predict the new number of customers perfectly.

In [0]:
import numpy.random as random
random.seed(3)
new_random_temps = random.randint(30, 101, 50)
new_temps_reshaped = new_random_temps.reshape(-1, 1)
new_perfect_customers = (3*new_random_temps + 10)

In [0]:
from sklearn.metrics import mean_squared_error
new_predictions = perfect_model.predict(new_temps_reshaped)
np.sqrt(mean_squared_error(new_perfect_customers, new_predictions))

4.1722837113136385e-14

So our model predicts our future data with almost no error as well.

In [0]:
import plotly.graph_objects as go
model_trace = go.Scatter(x = new_random_temps, y = new_predictions, name = 'trained model', mode = 'lines')
new_data_trace = go.Scatter(x = new_random_temps, y = new_perfect_customers, name = 'new data', mode = 'markers')
layout = {'yaxis': {'title': 'customers'}, 'xaxis': {'title': 'temperature'}}
go.Figure([new_data_trace, model_trace], layout = layout)

### Summary

In this lesson, we saw a couple things.

First, we saw how to generate a dataset.  To do this, we first created a list of random inputs, our temperatures.  Then we created the corresponding outputs by simply creating outputs that were a function of our inputs.

The second thing we saw was what occurs when we train our model on a dataset that perfectly follows a linear equation.  What happens is our model can perfectly discover the underlying model.  We saw this when we trained our model on our dataset.  It perfectly discovered the parameters underlying the data,  $customers = 3*temperature + 10$.  We also saw that saw that our model also predicted data it had not previously seen, so long as the outputs also followed the underlying formula.

We'll use this perfect scenario by way of comparison to see what occurs when we do not have perfect data.